## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-17-06-50-09 +0000,bbc,UK emissions would rise under Tory energy plan...,https://www.bbc.com/news/articles/cn97x3jy8w1o...
1,2025-10-17-06-36-03 +0000,nypost,Officials release cause of death for newlywed ...,https://nypost.com/2025/10/17/us-news/official...
2,2025-10-17-06-24-41 +0000,nyt,Trump Plans to Meet With Putin to Talk About a...,https://www.nytimes.com/2025/10/16/us/politics...
3,2025-10-17-06-21-20 +0000,nypost,Dem school board chair drops Senate bid after ...,https://nypost.com/2025/10/17/us-news/iowa-sch...
4,2025-10-17-06-20-46 +0000,bbc,Author of I Want To Die But I Want To Eat Tteo...,https://www.bbc.com/news/articles/cpq1dy8w3jwo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2305/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,63
242,new,24
357,will,17
263,he,14
269,china,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
151,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...,143
215,2025-10-16-17-33-15 +0000,nypost,Trump says he will meet with Putin in Budapest...,https://nypost.com/2025/10/16/us-news/trump-sa...,127
272,2025-10-16-14-49-58 +0000,nypost,Trump says CBS under owner David Ellison will ...,https://nypost.com/2025/10/16/media/trump-says...,125
101,2025-10-16-23-01-00 +0000,wsj,Trump Says He Will Meet With Putin in Budapest...,https://www.wsj.com/world/russia/trump-and-put...,124
278,2025-10-16-14-19-27 +0000,bbc,India casts doubt on Trump's claim Modi will s...,https://www.bbc.com/news/articles/c8eykdy0567o...,119


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
151,143,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...
372,64,2025-10-16-09-00-00 +0000,wsj,What Happens When Oil and Gas Wells Die? West ...,https://www.wsj.com/business/energy-oil/what-h...
31,60,2025-10-17-03-46-08 +0000,nypost,"Zohran Mamdani dodges attacks, straight answer...",https://nypost.com/2025/10/16/us-news/mamdani-...
181,55,2025-10-16-19-13-22 +0000,bbc,At least three dead after Kenya police open fi...,https://www.bbc.com/news/articles/cwyk1l65nqvo...
164,42,2025-10-16-20-21-13 +0000,nypost,Thune says ‘time’s right’ for Russia sanctions...,https://nypost.com/2025/10/16/us-news/thune-sa...
109,40,2025-10-16-22-31-31 +0000,startribune,"Prison for woman who admitted she was driving,...",https://www.startribune.com/prison-for-woman-a...
313,39,2025-10-16-12-11-11 +0000,wapo,Hamas took 251 hostages from Israel into Gaza....,https://www.washingtonpost.com/world/interacti...
283,38,2025-10-16-14-02-46 +0000,wapo,Do you need a better job? Answer these five qu...,https://www.washingtonpost.com/business/2025/1...
245,38,2025-10-16-16-00-00 +0000,wsj,An extended government shutdown raises the pro...,https://www.wsj.com/economy/central-banking/fe...
39,36,2025-10-17-02-56-02 +0000,nypost,2 young Ohio children charged with attempted m...,https://nypost.com/2025/10/16/us-news/2-ohio-c...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
